# LLM Chatbot

## LLM

In [3]:
# Libraries
## pytorch framework
import torch 
## AutoModelForCausalLM -> pretrained model
## AutoTokenizer -> tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer


c:\Users\alvar\miniconda3\envs\chatbot-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# I load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

## Chat loop

In [ ]:
# Chat loop
chat_history_ids = None
n_turns = 10
print("Start chatting with the bot (type 'quit' to stop)")

for _ in range(n_turns):
    # User input
    user_input = input(">> User: ")
    if user_input.lower() == "quit":
        break

    # Tokenize the input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Generate the response (from history if exists)
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )

    # Decode response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f">> Bot: {response}")


Start chatting with the bot (type 'quit' to stop)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


>> Bot: 3 3. 4 14
>> Bot: Q uitos es buenos, quien era seguido?


## Visual

## Chatbot step by step